In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "deepseek-ai/deepseek-coder-1.3b-base"  # small-ish
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [2]:
# Encode some text into tokens
text = "你好世界"
tokens = tokenizer.encode(text)
print(tokens)

# Decode back into text
decoded = tokenizer.decode(tokens)
print(decoded)

# Inspect how it split the text
print([tokenizer.decode([t]) for t in tokens])


[32013, 1367, 1248, 4986]
<｜begin▁of▁sentence｜>你好世界
['<｜begin▁of▁sentence｜>', '你', '好', '世界']


In [3]:
# convert ids -> token strings
tokens = [32013, 1367, 1248, 4986]
print(tokenizer.convert_ids_to_tokens(tokens))

# vocab size and specials
print("vocab size:", tokenizer.vocab_size)
print("all special tokens:", tokenizer.all_special_tokens)
print("special tokens map:", tokenizer.special_tokens_map)

# encode without adding special tokens (if you don't want BOS/EOS)
tokens = tokenizer.encode("你好世界", add_special_tokens=False)
print(tokens)

# show ID -> decoding for each id (what you already saw)
print([tokenizer.decode([t]) for t in tokens])

['<｜begin▁of▁sentence｜>', 'ä½ł', 'å¥½', 'ä¸ĸçķĮ']
vocab size: 32000
all special tokens: ['<｜begin▁of▁sentence｜>', '<｜end▁of▁sentence｜>']
special tokens map: {'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<｜end▁of▁sentence｜>', 'pad_token': '<｜end▁of▁sentence｜>'}
[1367, 1248, 4986]
['你', '好', '世界']


In [4]:
with open("再别康桥.txt", "r") as f:
    text = f.read()
tokens = tokenizer.encode(text)
for t in tokens:
    print("token id:", t, "token str:", tokenizer.decode([t]))
print("length of tokens:", len(tokens))

token id: 32013 token str: <｜begin▁of▁sentence｜>
token id: 4642 token str: 轻
token id: 4642 token str: 轻
token id: 337 token str: 的
token id: 848 token str: 我
token id: 26388 token str: 走了
token id: 19385 token str: ，
token id: 185 token str: 

token id: 2198 token str: 正
token id: 1410 token str: 如
token id: 848 token str: 我
token id: 4642 token str: 轻
token id: 4642 token str: 轻
token id: 337 token str: 的
token id: 908 token str: 来
token id: 1989 token str: ；
token id: 185 token str: 

token id: 848 token str: 我
token id: 4642 token str: 轻
token id: 4642 token str: 轻
token id: 337 token str: 的
token id: 6716 token str: 招
token id: 1897 token str: 手
token id: 19385 token str: ，
token id: 185 token str: 

token id: 1147 token str: 作
token id: 2501 token str: 别
token id: 2787 token str: 西
token id: 17881 token str: 天的
token id: 5973 token str: 云
token id: 7817 token str: 彩
token id: 397 token str: 。
token id: 185 token str: 

token id: 185 token str: 

token id: 1865 token str: 那
token 

In [5]:
from transformers import AutoTokenizer

tokenizer_gemma = AutoTokenizer.from_pretrained("google/gemma-2b")

In [6]:
tokens = tokenizer_gemma.encode(text)
print("vocab size:", tokenizer_gemma.vocab_size)
for t in tokens:
    print("token id:", t, "token str:", tokenizer_gemma.decode([t]))
print("length of tokens:", len(tokens))

vocab size: 256000
token id: 2 token str: <bos>
token id: 79424 token str: 轻轻
token id: 153698 token str: 的我
token id: 44913 token str: 走了
token id: 235365 token str: ，
token id: 108 token str: 

token id: 161181 token str: 正如
token id: 235509 token str: 我
token id: 79424 token str: 轻轻
token id: 235370 token str: 的
token id: 235547 token str: 来
token id: 236334 token str: ；
token id: 108 token str: 

token id: 235509 token str: 我
token id: 79424 token str: 轻轻
token id: 235370 token str: 的
token id: 237219 token str: 招
token id: 235616 token str: 手
token id: 235365 token str: ，
token id: 108 token str: 

token id: 235591 token str: 作
token id: 236273 token str: 别
token id: 235990 token str: 西
token id: 55881 token str: 天的
token id: 236537 token str: 云
token id: 236729 token str: 彩
token id: 235362 token str: 。
token id: 109 token str: 


token id: 235779 token str: 那
token id: 236811 token str: 河
token id: 241225 token str: 畔
token id: 172765 token str: 的金
token id: 238110 token str: 柳


In [3]:
import torch
# Example with DeepSeek

# Your input so far (the context)
inputs = tokenizer(text, return_tensors="pt")

# Run the model
with torch.no_grad():
    outputs = model(**inputs)

# Get logits for the next token prediction
logits = outputs.logits[0, -1, :]  # last token's logits
print(logits.shape)  # should be (vocab_size,)
# print top 10 logits
print(torch.topk(logits, 10))
probs = torch.softmax(logits, dim=-1)

# Find top probable tokens
top_probs, top_indices = torch.topk(probs, 10)
for i, p in zip(top_indices, top_probs):
    print(f"{tokenizer.decode(i)}: {p.item():.5f}")


NameError: name 'tokenizer' is not defined

In [19]:
# test on this alphabet:
alphabet = ["<EOF>", "a", "e", "i", "o", "u"]
probabilities = [0.1, 0.2, 0.3, 0.15, 0.15, 0.1]
symbol_to_id = {s: i for i, s in enumerate(alphabet)}
message = ["u", "o", "o", "e", "u", "i", "a", "<EOF>"]

import encoder

enc = encoder.Encoder()
for symbol in message:
    id = symbol_to_id[symbol]
    enc.encode(symbol, id, probabilities)
print(enc.get_encoded_bytes())  # print as integer






b'\xfc\xd1\xa7'


In [2]:
from encoder import Encoder
from decoder import Decoder
from arithmetic_coding import Coder
from bitReadWrite import BitReader, BitWriter

# Jupyter test cell: end-to-end encode/decode using your Coder / Encoder / Decoder classes.
# Paste this cell below where you have defined or imported Coder, Encoder, Decoder.

from typing import List
import bisect

# -------------------------
# Small helpers (copy into cell so test is self-contained)
# -------------------------
def probs_to_counts_largest_remainder(probs: List[float], slots: int) -> List[int]:
    """Deterministic mapping of floats -> integer counts summing to exactly 'slots'."""
    if len(probs) == 0:
        raise ValueError("Empty probability list")
    raw = [float(p) for p in probs]
    s = sum(raw)
    if s <= 0:
        raise ValueError("Probabilities must sum to > 0")
    scaled = [r * slots / s for r in raw]
    floors = [int(x) for x in scaled]
    remainder = slots - sum(floors)
    fracs = sorted(((i, scaled[i] - floors[i]) for i in range(len(probs))),
                   key=lambda x: x[1], reverse=True)
    i = 0
    while remainder > 0 and i < len(fracs):
        floors[fracs[i][0]] += 1
        remainder -= 1
        i += 1
    # safety: if a nonzero prob got zero count we must increase slots/precision
    for p, c in zip(raw, floors):
        if p > 0.0 and c == 0:
            raise ValueError("Precision too low: a nonzero probability mapped to 0 count")
    return floors

def counts_to_cum_desc(counts: List[int]) -> List[int]:
    """Convert counts -> descending cumulative array used by CACM: cum_desc[0]=total, cum_desc[-1]=0."""
    total = sum(counts)
    cum = [total]
    s = 0
    for c in counts:
        s += c
        cum.append(total - s)
    return cum  # length n+1, cum[0]=total, cum[n]=0


# -------------------------
# Test config (your example)
# -------------------------
alphabet = ['a', 'e', 'i', 'o', 'u', '<EOF>']
probs = [0.1, 0.2, 0.3, 0.15, 0.15, 0.1]
message = ["u", "o", "o", "e", "u", "i", "a", "<EOF>"]

# -------------------------
# Ensure Coder / Encoder / Decoder exist (user's classes)
# -------------------------
if 'Coder' not in globals():
    raise RuntimeError("Coder class not found in notebook scope. Define/import Coder before running this cell.")
if 'Encoder' not in globals():
    raise RuntimeError("Encoder class not found in notebook scope. Define/import Encoder before running this cell.")
if 'Decoder' not in globals():
    raise RuntimeError("Decoder class not found in notebook scope. Define/import Decoder before running this cell.")

# -------------------------
# Build model counts and cumulative (descending) expected by textbook wrappers
# -------------------------
# instantiate a temporary Coder to choose slots consistent with its precision
temp_coder = globals()['Coder'](b=16)   # choose b=16 for compactness; increase to 32 to match paper if desired
# slots: choose temp_coder.tb ( = 2^b - 1 ) so total fits inside coder precision
slots = temp_coder.tb
counts = probs_to_counts_largest_remainder(probs, slots)
cum_desc = counts_to_cum_desc(counts)

print("Model counts (first few):", counts[:6], "sum:", sum(counts))
print("cum_desc[0] (total):", cum_desc[0])

# -------------------------
# ENCODING (use Coder.storeRegion directly to avoid wrapper convention mismatch)
# -------------------------
coder_enc = globals()['Coder'](b=16)
bw = BitWriter()

# prime the encoder (your Coder method is start_encode)
if hasattr(coder_enc, 'start_encode'):
    coder_enc.start_encode(bw)
else:
    raise RuntimeError("Coder missing start_encode; adapt call to your API.")

sym_to_idx = {s: i for i, s in enumerate(alphabet)}

# perform symbol-by-symbol encoding using textbook descending cum_desc convention
for sym in message:
    idx = sym_to_idx[sym]
    l = cum_desc[idx + 1]
    h = cum_desc[idx]
    # call the exact method you defined earlier
    coder_enc.storeRegion(l, h)

# finalize encoder and flush bit-writer
coder_enc.finish_encode()
bw.flush(padbit=0)   # your BitWriter implementation expects padbit param
encoded_bytes = bw.getvalue()
print("Encoded bytes (hex):", encoded_bytes.hex())
print("Encoded bitstring (bytes*8 bits shown):", ''.join(f'{b:08b}' for b in encoded_bytes))

# diagnostic: print coder_enc internal state after encoding finishes
print("After encoding: encoder L=", coder_enc.L, "R=", coder_enc.R, "bits_waiting=", coder_enc.bits_waiting)
# prime a new decoder and print its initial state
coder_dec = globals()['Coder'](b=16)
coder_dec.start_decode(BitReader(encoded_bytes))
print("Decoder primed: L=", coder_dec.L, "D=", coder_dec.D, "R=", coder_dec.R)


# -------------------------
# DECODING (direct Coder usage — uses attributes L,R,D and loadRegion)
# -------------------------
coder_dec = globals()['Coder'](b=16)
br = BitReader(encoded_bytes)

# prime decoder
if hasattr(coder_dec, 'start_decode'):
    coder_dec.start_decode(br)
else:
    raise RuntimeError("Coder missing start_decode; adapt call to your API.")

decoded = []
for i in range(len(message)):
    # read the current integer range and target using the Coder attributes
    R = coder_dec.R
    target = coder_dec.D - coder_dec.L
    total = cum_desc[0]

    # defensive checks
    if R <= 0:
        raise RuntimeError(f"Decoder: invalid R={R} at step {i}")
    if not (0 <= target < R):
        raise RuntimeError(f"Decoder invalid target={target} at step {i} (L={coder_dec.L}, D={coder_dec.D}, R={R})")

    # textbook mapping to integer count
    count = ((target + 1) * total - 1) // R

    # find symbol s with cum_desc[s+1] <= count < cum_desc[s]
    s_found = None
    for s in range(len(cum_desc) - 1):
        if cum_desc[s + 1] <= count < cum_desc[s]:
            s_found = s
            break
    if s_found is None:
        s_found = len(cum_desc) - 2

    # update coder state by loading the symbol's region
    l = cum_desc[s_found + 1]
    h = cum_desc[s_found]
    coder_dec.loadRegion(l, h)

    decoded.append(alphabet[s_found])
    print(f"decoded step {i}: idx={s_found}, symbol={alphabet[s_found]}")

print("Decoded message:", decoded)
print("Round-trip OK?", decoded == message)


Model counts (first few): [6554, 13107, 19661, 9830, 9830, 6553] sum: 65535
cum_desc[0] (total): 65535
Encoded bytes (hex): 5400
Encoded bitstring (bytes*8 bits shown): 0101010000000000
After encoding: encoder L= 0 R= 32768 bits_waiting= 0
Decoder primed: L= 0 D= 21504 R= 65536
decoded step 0: idx=3, symbol=o
decoded step 1: idx=2, symbol=i


RuntimeError: Decoder invalid target=-15971 at step 2 (L=58979, D=43008, R=19661)

In [12]:
# Deterministic interval-compare diagnostic
from typing import List

# reuse your model helpers
def probs_to_counts_largest_remainder(probs: List[float], slots: int) -> List[int]:
    raw = [float(p) for p in probs]; s = sum(raw)
    scaled = [r * slots / s for r in raw]
    floors = [int(x) for x in scaled]; remainder = slots - sum(floors)
    fracs = sorted(((i, scaled[i] - floors[i]) for i in range(len(probs))),
                   key=lambda x: x[1], reverse=True)
    i = 0
    while remainder > 0 and i < len(fracs):
        floors[fracs[i][0]] += 1; remainder -= 1; i += 1
    return floors

def counts_to_cum_desc(counts: List[int]) -> List[int]:
    total = sum(counts); cum=[total]; s=0
    for c in counts:
        s+=c; cum.append(total-s)
    return cum

# config
alphabet = ['a','e','i','o','u','<EOF>']
probs = [0.1,0.2,0.3,0.15,0.15,0.1]
message = ["u","o","o","e","u","i","a","<EOF>"]

# model
temp = Coder(b=16)
slots = temp.tb
counts = probs_to_counts_largest_remainder(probs, slots)
cum_desc = counts_to_cum_desc(counts)
total = cum_desc[0]
print("counts:", counts)
print("cum_desc:", cum_desc)
print("encoded message length:", len(message))

# ENCODE and record encoder absolute intervals (new_low,new_high) BEFORE renormalisation
coder_enc = Coder(b=16)
bw = BitWriter()
coder_enc.start_encode(bw)
mask = coder_enc.mask

enc_intervals = []  # list of tuples (new_low, new_high, symbol)
print("\nENCODING: record intervals (before renorm)")
for i,sym in enumerate(message):
    idx = alphabet.index(sym)
    l = cum_desc[idx+1]; h = cum_desc[idx]
    lower = total - h; upper = total - l
    rng = coder_enc.R
    new_low = coder_enc.L + (rng * lower) // total
    new_high = coder_enc.L + (rng * upper) // total - 1
    enc_intervals.append((new_low, new_high, sym))
    # apply to coder and renormalise (as you do)
    coder_enc.L = new_low & mask
    coder_enc.R = ((new_high - new_low + 1) & mask)
    coder_enc._output_bits()
    print(f"enc[{i}] '{sym}': lower={lower} upper={upper} -> new_low={new_low} new_high={new_high}  (post-renorm L={coder_enc.L} R={coder_enc.R} bits_waiting={coder_enc.bits_waiting})")

coder_enc.finish_encode()
bw.flush(padbit=0)
encoded = bw.getvalue()
print("\nEncoded bytes (hex):", encoded.hex())
print("Encoded bitstring:", ''.join(f"{b:08b}" for b in encoded))
print("encoder final L,R:", coder_enc.L, coder_enc.R)

# DECODE and compare
coder_dec = Coder(b=16)
br = BitReader(encoded)
coder_dec.start_decode(br)
print("\nDECODING: compare intervals vs encoder records")
decoded = []
for i in range(len(message)):
    L = coder_dec.L; R = coder_dec.R; D = coder_dec.D
    print(f"\nstep {i}: before selection: L={L} D={D} R={R}")
    # compute every symbol's absolute interval as encoder would before renorm
    found_s = None
    candidate_intervals = []
    for s in range(len(cum_desc)-1):
        l = cum_desc[s+1]; h = cum_desc[s]
        lower = total - h; upper = total - l
        new_low_s = L + (R * lower) // total
        new_high_s = L + (R * upper) // total - 1
        candidate_intervals.append((s, alphabet[s], new_low_s, new_high_s))
        if new_low_s <= D <= new_high_s:
            found_s = s
    # report candidates
    print("  candidate intervals (s, sym, new_low, new_high):")
    for c in candidate_intervals:
        print("   ", c)
    if found_s is None:
        print("  *** DECODER: no candidate interval contains D! ***")
    else:
        print(f"  DECODER selects s={found_s} sym='{alphabet[found_s]}' interval=({candidate_intervals[found_s][2]}, {candidate_intervals[found_s][3]})")
    # compare to encoder's recorded interval for this position
    enc_low, enc_high, enc_sym = enc_intervals[i]
    print("  ENCODER recorded interval:", (enc_low, enc_high, enc_sym))
    if found_s is None or enc_sym != alphabet[found_s] or (enc_low != candidate_intervals[found_s][2] or enc_high != candidate_intervals[found_s][3]):
        print("\nMISMATCH FOUND at step", i)
        print("  encoder interval vs decoder candidate:")
        print("   encoder:", (enc_low, enc_high, enc_sym))
        if found_s is None:
            print("   decoder: no containing interval found (D not in any)")
        else:
            print("   decoder candidate:", candidate_intervals[found_s])
        # show a bit more state and stop
        print("  encoder post-renorm state for this step (L,R,bits_waiting):", coder_enc.L, coder_enc.R, coder_enc.bits_waiting)
        print("  decoder pre-renorm state for this step (L,D,R):", L, D, R)
        raise SystemExit("Stop: mismatch — paste the above and I'll recommend the single-line fix.")
    # If consistent, perform decoder load/renorm exactly as encoder did
    # compute decoder's new_low/new_high identical to encoder's formulas
    l = cum_desc[found_s+1]; h = cum_desc[found_s]
    lower = total - h; upper = total - l
    new_low = coder_dec.L + (coder_dec.R * lower) // total
    new_high = coder_dec.L + (coder_dec.R * upper) // total - 1
    coder_dec.L = new_low & coder_dec.mask
    coder_dec.R = ((new_high - new_low + 1) & coder_dec.mask)
    coder_dec._discard_bits()
    decoded.append(alphabet[found_s])
    print(f"  accepted symbol '{alphabet[found_s]}', decoder after renorm: L={coder_dec.L}, D={coder_dec.D}, R={coder_dec.R}")

print("\nDecoded sequence:", decoded)
print("Roundtrip OK?:", decoded == message)


counts: [6554, 13107, 19661, 9830, 9830, 6553]
cum_desc: [65535, 58981, 45874, 26213, 16383, 6553, 0]
encoded message length: 8

ENCODING: record intervals (before renorm)
enc[0] 'u': lower=49152 upper=58982 -> new_low=49152 new_high=58981  (post-renorm L=32768 R=19660 bits_waiting=0)
enc[1] 'o': lower=39322 upper=49152 -> new_low=44564 new_high=47512  (post-renorm L=28832 R=23592 bits_waiting=0)
enc[2] 'o': lower=39322 upper=49152 -> new_low=42987 new_high=46525  (post-renorm L=16216 R=28312 bits_waiting=0)
enc[3] 'e': lower=6554 upper=19661 -> new_low=19047 new_high=24708  (post-renorm L=10652 R=22648 bits_waiting=0)
enc[4] 'u': lower=49152 upper=58982 -> new_low=27638 new_high=31034  (post-renorm L=24496 R=27176 bits_waiting=0)
enc[5] 'i': lower=19661 upper=39322 -> new_low=32649 new_high=40801  (post-renorm L=32292 R=32612 bits_waiting=2)
enc[6] 'a': lower=0 upper=6554 -> new_low=32292 new_high=35552  (post-renorm L=28960 R=26088 bits_waiting=5)
enc[7] '<EOF>': lower=58982 upper=65

In [1]:
from utils import probs_to_counts_largest_remainder, counts_to_cum_desc
from arithmetic_coding import Coder
from encoder import Encoder
from decoder import Decoder
from bitReadWrite import BitReader, BitWriter

alphabet = ['a', 'e', 'i', 'o', 'u', '<EOF>']
probs = [0.1, 0.2, 0.3, 0.15, 0.15, 0.1]
message = ["u", "o", "o", "e", "u", "i", "a", "<EOF>"]

# choose precision b=16 for quickness
coder_temp = Coder(b=16)
slots = coder_temp.tb  # = 2^b - 1
counts = probs_to_counts_largest_remainder(probs, slots)
cum_desc = counts_to_cum_desc(counts)

print("Model counts:", counts, "total:", cum_desc[0])

# encode
cw = BitWriter()
coder_enc = Coder(b=16)
enc = Encoder(coder_enc, cw)
sym_to_idx = {s: i for i, s in enumerate(alphabet)}
for sym in message:
    idx = sym_to_idx[sym]
    enc.encode_symbol(idx, cum_desc)
enc.finish()
cw.flush(padbit=0)
encoded = cw.getvalue()
print("Encoded bytes (hex):", encoded.hex())

# decode
br = BitReader(encoded)
coder_dec = Coder(b=16)
dec = Decoder(coder_dec, br)
decoded = []
for _ in range(len(message)):
    idx = dec.decode_symbol(cum_desc)
    decoded.append(alphabet[idx])
print("Decoded:", decoded)
print("Round-trip OK?", decoded == message)

Model counts: [6554, 13107, 19661, 9830, 9830, 6553] total: 65535
Encoded bytes (hex): dab828
Decoded: ['u', 'o', 'o', 'e', 'u', 'i', 'a', '<EOF>']
Round-trip OK? True
